In [1]:
import numpy as np
import numexpr as ne
from numba import njit
import matplotlib.pyplot as plt
import matplotlib.animation as animation
%matplotlib qt5
%reload_ext autoreload
%autoreload 2

import verlet as vl


init_row_len = 32
init_col_len = 32
num_objs = init_row_len * init_col_len
masses = np.ones(num_objs)
walls = np.array([(-8, 8), (-8, 8), (-8, 8)])
x_pos_offset = (walls[0, 1] - walls[0, 0])/(init_row_len + 1)/2
y_pos_offset = (walls[1, 1] - walls[1, 0])/(init_col_len + 1)/2
rand_offset_factor = 0.1
vel_init = 0
initial_positions = np.zeros((num_objs, 3))
initial_velocities = np.zeros((num_objs, 3))
initial_positions[:, 0:2] = \
    np.stack(
        np.meshgrid(
            np.linspace(walls[0, 0] + 0*x_pos_offset, 
                        walls[0, 1] - 0*x_pos_offset, init_row_len), 
            np.linspace(walls[1, 0] + 0*y_pos_offset, 
                        walls[1, 1] - 0*y_pos_offset, init_col_len)), 2
    ).reshape(-1, 2) + np.hstack((
        np.random.uniform(
            -x_pos_offset * rand_offset_factor, 
            x_pos_offset * rand_offset_factor, 
            (num_objs, 1)
        ), np.random.uniform(
            -y_pos_offset * rand_offset_factor, 
            y_pos_offset * rand_offset_factor, 
            (num_objs, 1))))
initial_velocities[:, 0:2] = np.random.normal(0, vel_init, (num_objs, 2)) #np.array([[vel_init, 0]]).repeat(num_objs, 0)
 
total_time = 1
dt = 0.0001

def accel(mass, position, velocity, dt=dt):
    accel1 = vl.accel_intermolecular(
        mass=mass, position=position, velocity=velocity, V_LJ=1, r_m=1
    ) 
    accel2 = vl.accel_softwalls(
        mass=mass, position=position, velocity=velocity, dt=dt, 
        dt_per_period=8, walls=((-8, 8), (-8, 8), (-8, 8))
    )
    return accel1 + accel2

positions_over_time, velocities_over_time = vl.integrate_verlet(
    masses, initial_positions, initial_velocities, 
    total_time, dt)
num_steps = positions_over_time.shape[0]
steps = np.arange(num_steps)
times = steps * dt

  0%|          | 0/10001 [00:00<?, ?it/s]

In [2]:
k_B = 1.380649e-23
V_LJ_to_Joules = 1.65e-21
temperature = (
    masses * (velocities_over_time**2).sum(axis=2)
).mean(axis=1) * 0.5 * V_LJ_to_Joules * 2 / 3 / k_B

In [3]:
fig, ax = plt.subplots(1, 1, figsize=(8, 8))
cmap = plt.cm.CMRmap
norm = plt.Normalize(vmin=0, vmax=(num_objs - 1) * 1.5)
min_step_to_plot = int(0.5 * total_time / dt)
max_step_to_plot = int(0.6 * total_time / dt)
for step in steps[min_step_to_plot:max_step_to_plot:1]:
    ax.scatter(
        positions_over_time[step, :, 0], 
        positions_over_time[step, :, 1], 
        color=cmap(norm(np.arange(num_objs))),
        alpha=(
            (step - min_step_to_plot)/(max_step_to_plot - min_step_to_plot)
        ) * 0.9 + 0.1)
ax.axvline(-8)
ax.axvline(8)
ax.axhline(-8)
ax.axhline(8)
ax.set_xlim(-9, 9)
ax.set_ylim(-9, 9)
ax.set_aspect('equal')
fig.show()

In [5]:
fig2, ax2 = plt.subplots(1, 2, figsize=(16, 8))

ax2[0].set_xlim(-9, 9)
ax2[0].set_ylim(-9, 9)
ax2[0].set_aspect('equal')  
ax2[0].axvline(-8)
ax2[0].axvline(8)
ax2[0].axhline(-8)
ax2[0].axhline(8)
scat = ax2[0].scatter(positions_over_time[0, :, 0], 
                positions_over_time[0, :, 1], 
                color=cmap(norm(np.arange(num_objs))),
                #alpha=step/max_step_to_plot)
)

ax2[1].set_xlim(0, (num_steps + 1) * dt)
ax2[1].set_ylim(0, temperature.max() * 1.05)
line2, = ax2[1].plot(times[:1], temperature[:1], zorder=1)
scat2 = ax2[1].scatter(times[0], temperature[0], color='orange', zorder=2)

def update_plot(i):
    scat.set_offsets(np.c_[positions_over_time[i, :, 0], 
                positions_over_time[i, :, 1]])
    line2.set_data(times[:i+1], temperature[:i+1])
    scat2.set_offsets(np.c_[times[i], temperature[i]])
                #, 
                #color=cmap(norm(np.arange(num_objs))),
                #alpha=step/max_step_to_plot)
    

ani = animation.FuncAnimation(fig2, update_plot, 
                              frames=range(0, int(num_steps), 10),
                              interval=1000/60)
ani.save('sim.gif', writer='ffmpeg', fps=60)
fig2.show()